In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import spacy
from pathlib import Path

# Read in the test data for imgs
output_dir = Path("./models")
print("Loading spacy model from ", output_dir)
nlp = spacy.load(output_dir)
print("Finished Loading")

Loading spacy model from  models
Finished Loading


In [137]:
img_test_path = "./ocr_all.csv"

TEST_DATA = []
with open(img_test_path, 'r') as file:
    lines = [line.strip() for line in file.readlines()]
    
    # Annotate the data
    for line in lines[1:]:
        doc = nlp(line)
        content = line
        entities = {"entities": []}
        for ent in doc.ents:
            entities["entities"].append((ent.label_, ent.text))
        
        v = (content, entities)
        TEST_DATA.append(v)

In [138]:
# with open ('Data/spacy.txt', 'rb') as fp:
#     TEST_DATA = pickle.load(fp)
    
row_labels = []
for content, obj in TEST_DATA:
    entities = obj["entities"]
    labels = {
        "totalAmount": [],
        "totalLabel": [],
        "date": [],
        "address": [],
        "vendor": []
    }
    
    docId = content.split(",")[1]
    
    for entity in entities:
        #p = (int(entity[0]), int(entity[1]))
        #key = entity[2]
        #value = content[p[0]:p[1]].replace("$", "").strip()
        key = entity[0]
        value = entity[1]
        labels[key].append((docId, value))
    
    row_labels.append(labels)
    
bucket = {
        "amount": [],
        "totalLabel": [],
        "date": [],
        "vendor_address": [],
        "vendor_name": []
}

keyMap = {
    "totalAmount": "amount",
    "vendor": "vendor_name",
    "totalLabel": "totalLabel",
    "date": "date",
    "address": "vendor_address"
}

for item in row_labels:
    for k, v in item.items():
        bucket[keyMap[k]].extend(v)

In [139]:
import re

# Filter for numeric amounts
bucket["amount"] = list(filter(lambda x: re.match("\d+[.]\d+", x[1]), bucket["amount"]))

In [140]:
from sentence_transformers import SentenceTransformer

# Vectorize all of the fields
dataframes = []
model = SentenceTransformer('bert-base-nli-mean-tokens')

for k in bucket.keys():
    ids = [v[0] for v in bucket[k]]
    values = [v[1] for v in bucket[k]]
    
    entity_embeddings = model.encode(values)
    
    obj = {
        "documentid": ids, 
    }
    obj[k] = values
    if k != "amount":
        obj[k + "_embedding"] = [(e) for e in model.encode(values)]
    else:
        obj[k + "_embedding"] = [(np.array([float(e)])) for e in values]
    
    df = pd.DataFrame(obj)
    dataframes.append(df)

In [143]:
# Merge all the dfs together
merge = pd.merge(dataframes[0],dataframes[2],on='documentid',how='outer')
merge = pd.merge(merge,dataframes[3],on='documentid',how='outer')
merge = pd.merge(merge,dataframes[4],on='documentid',how='outer')
img_df = merge
img_df

,documentid,amount,amount_embedding,date,date_embedding,vendor_address,vendor_address_embedding,vendor_name,vendor_name_embedding
0,00d0243046961,9.65,[9.65],29/03/18,"[0.5429581, -0.0665234, 2.096464, 0.3384345, 0...",BOOK CO. (M) SDN BHD,"[-0.30416343, 0.39372644, 1.096122, -0.1557415...",NaN,NaN
1,00d0243046961,9.65,[9.65],29/03/18,"[0.5429581, -0.0665234, 2.096464, 0.3384345, 0...","NO 8, JALAN 7/118B, DESA TUN RAZAK 56000 KUALA...","[-0.08186542, 0.43657455, 0.7416691, 0.2938639...",NaN,NaN
2,00d0164592408,6.00,[6.0],05-03-2018,"[0.16170992, 0.006006032, 2.6065888, 0.2249980...","NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR...","[0.17609178, 0.24673975, 0.31823435, 0.2454735...",NaN,NaN
3,00d0696518680,7.00,[7.0],03/02/2018,"[0.5998782, 0.042366665, 2.5505097, 0.5825793,...",NO 7. SIMPANG OFF BATU VILLAGE. JALAN IPOH BAT...,"[-0.008996537, 0.60043174, 0.5059461, 0.335876...",SENG THO HARDWARE TRADING,"[0.5843093, 1.0965058, 1.344565, 0.30423525, 0..."
4,00d0142736377,35.00,[35.0],13-06-2018,"[-0.008761449, 0.13843873, 2.4405327, 0.082088...","NO.1, TAMAN SRI DENGKIL, JALAN AIR HITAM 43800...","[0.061991673, 0.8515016, 0.2755021, 0.21966623...",THREE SEAFOOD RESTAURANT SDN BHD,"[0.3343588, 0.7696248, 0.338173, -0.6639876, 0..."
...,...,...,...,...,...,...,...,...,...
661,00d0531848355,NaN,NaN,NaN,NaN,"12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAM...","[0.49348155, 0.7787099, 0.49291992, -0.2070064...",SDN BHD 18 JUN 2018,"[0.9588894, 0.36526513, 2.5297034, -0.04193136..."
662,00d0147748424,NaN,NaN,NaN,NaN,"12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAM...","[0.49348173, 0.77871, 0.49292004, -0.20700623,...",INTERNATIONAL SDN BHD 11 JUN 2018,"[0.62450725, 0.40023202, 1.8833292, 0.06522257..."
663,00d0178057064,NaN,NaN,NaN,NaN,"NO.1, JALAN PERMAS 10/5, BANDAR BARU PERMAS JA...","[-0.031045718, 1.0091283, 0.015718937, -0.2923...",CAKE HOUSE SDN BHD 862725-U,"[0.105295576, 1.6263118, 0.78402436, -0.367346..."
664,00d0961798581,NaN,NaN,NaN,NaN,"LOT 2942 & 2945, JLN SERI SENTOSA 8, SERI SENT...","[0.0834254, 0.30150214, 1.0114563, 0.38561654,...",CHECKERS HYPERMARKET SDN BHD,"[0.2789244, 0.7458712, 1.4719939, 0.25180003, ..."


In [144]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg

dist_cols = ["image_doc_id", "user_doc_id", "amount_dist", "vendor_name_dist", "vendor_address_dist", "date_dist"]
cols = ["amount_embedding", "vendor_name_embedding", "vendor_address_embedding", "date_embedding"]

def getEmbedding(x):
    x_row = x.copy()
    cols = ['date', 'vendor_name','vendor_address']
    for col in cols:
        # Create a model for each col and make the embedding
        values = x_row[col]
        embeddings = model.encode(values)
        x_row[col + "_embedding"] = [(e) for e in embeddings]

    # The embeddings for amounts are just the amounts
    x_row["amount_embedding"] = [(np.array(float(x["amount"])))]
    return x_row

def getImageDists(x):
    # Get the embedding for the x value
    ref = getEmbedding(x)
    
    df = pd.DataFrame(columns=dist_cols)
    
    for i, row in img_df.iterrows():
        img_id = row["documentid"]
        user_id = ref['documentid']
        entry = {
           "image_doc_id": img_id,
            "user_doc_id": user_id 
        }
        
        for col in cols:
            value_col = col.rsplit("_", 1)[0]
            embed_col = col
                      

            dist = linalg.norm(np.array(ref[embed_col]) - np.array(row[embed_col]))
            
            entry[value_col + "_img"] = row[value_col]
            entry[value_col + "_user"] = ref[value_col]
            
            entry[value_col + "_dist"] = dist
        
        df = df.append(entry, ignore_index=True)
        
    return df

In [145]:
def match(x, n=1, w=[1, 1, 1, 1]):
    dists = getImageDists(x)
    dists["sum"] = (w[0] * dists["amount_dist"]) + (w[1] * dists["vendor_name_dist"]) + (w[2] * dists["vendor_address_dist"]) + (w[3] * dists["date_dist"])
    dists = dists.sort_values(["sum"])
    return dists.head(n)

In [ ]:
from tqdm import tqdm

test = pd.read_csv("./test_transactions.csv")

t = 0
n = len(test)
tested = .0001

bar = tqdm(test.iterrows())
for i, row in bar:
    res = match(row, 3, [1, 1, 1, 1])
    tested += 1
        
    if (res["image_doc_id"].values[0] == res["user_doc_id"].values[0]):
        print("match")
        print(res)
        t += 1
    else:
        print("no match")
        print(res)
        #print(m[m["image_doc_id"] == res["user_doc_id"]])
        if res["user_doc_id"].values[0] not in img_df["documentid"].values:
            n -= 1
            print(res["user_doc_id"], "not in imgs")
            tested -= 1
            
    print("Running Adj Acc:", ( t / tested))
        
        
        
print("Total Acc:", t, "/", len(test))
print("Adj. Acc:", t, "/", n)

1it [00:04,  4.15s/it]

no match
      image_doc_id    user_doc_id  amount_dist  vendor_name_dist  \
455  00d0243521534  00d0472780579          2.5         10.893335   
456  00d0243521534  00d0472780579          2.5         10.893335   
127  00d0340785534  00d0472780579          0.3         13.178209   

     vendor_address_dist  date_dist amount_img  amount_user    date_img  \
455             8.329150  11.646407       5.30          7.8  19/03/2018   
456             8.329150  11.646407       5.30          7.8  19/03/2018   
127             7.687895  12.365294       8.10          7.8    17-06-18   

    date_user                                 vendor_address_img  \
455  2018-4-4  NO.2, GROUND FLOOR, JALAN REKO SENTRAL 8 TAMAN...   
456  2018-4-4  NO.2, GROUND FLOOR, JALAN REKO SENTRAL 8 TAMAN...   
127  2018-4-4  LOT 1851-A & 1851-B, JALAN KP8 6, KAWASAN PERI...   

                                   vendor_address_user        vendor_name_img  \
455  NO.2, JALAN TEMENGUNG 19/9, SEKSYEM 9, BANDAR ...  RESTORA

2it [00:07,  3.88s/it]

match
      image_doc_id    user_doc_id  amount_dist  vendor_name_dist  \
562  00d0269258457  00d0269258457          0.1         10.437742   
307  00d0552375573  00d0269258457          3.9         10.437742   
316  00d0714317145  00d0269258457          5.9         10.437742   

     vendor_address_dist  date_dist amount_img  amount_user    date_img  \
562             1.008541   9.610733      24.00         24.1  15/09/2017   
307             1.008541   9.827070      28.00         24.1  04/09/2017   
316             1.008541   8.934736      30.00         24.1  12/09/2017   

     date_user                                 vendor_address_img  \
562  2017-9-13  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   
307  2017-9-13  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   
316  2017-9-13  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   

                                   vendor_address_user  \
562  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   
307  NO. 17-G, JALAN SETIA 

3it [00:11,  3.59s/it]

...

31it [01:33,  2.84s/it]

no match
      image_doc_id    user_doc_id  amount_dist  vendor_name_dist  \
455  00d0243521534  00d0741622526          0.3          9.871409   
456  00d0243521534  00d0741622526          0.3          9.871409   
474  00d0106074517  00d0741622526          0.0         13.035134   

     vendor_address_dist  date_dist amount_img  amount_user    date_img  \
455             8.288689  12.049611       5.30          5.0  19/03/2018   
456             8.288689  12.049611       5.30          5.0  19/03/2018   
474             8.268308  13.282076       5.00          5.0    30-04-18   

    date_user                                 vendor_address_img  \
455  2018-6-5  NO.2, GROUND FLOOR, JALAN REKO SENTRAL 8 TAMAN...   
456  2018-6-5  NO.2, GROUND FLOOR, JALAN REKO SENTRAL 8 TAMAN...   
474  2018-6-5  LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERI...   

                                   vendor_address_user        vendor_name_img  \
455  NO.2, JALAN TEMENGGUNG 19/9, SEKSGYEN 9, BANDA...  RESTORA

32it [01:36,  2.87s/it]

match
      image_doc_id    user_doc_id  amount_dist  vendor_name_dist  \
342  00d0204688228  00d0204688228          0.0          9.095043   
149  00d0564199588  00d0204688228          1.5          9.095043   
468  00d0923796657  00d0204688228          2.0          9.095043   

     vendor_address_dist  date_dist amount_img  amount_user    date_img  \
342             1.008541   7.251463       4.00          4.0  13/07/2017   
149             1.008541   9.302861       2.50          4.0  25/12/2017   
468             1.008541   9.403475       6.00          4.0  30/03/2017   

     date_user                                 vendor_address_img  \
342  2017-7-13  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   
149  2017-7-13  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   
468  2017-7-13  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   

                                   vendor_address_user  \
342  NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...   
149  NO. 17-G, JALAN SETIA 

33it [01:39,  2.84s/it]

In [79]:
matchs(test.iloc[9]).to_csv("Model_Example.csv")